In [1]:
!pip install -q kaggle

In [3]:
from google.colab import files

In [4]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jingyuechen","key":"f9dabfc4cd2d26ed5d2cdfc6537008da"}'}

In [6]:
! mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets list

ref                                                               title                                                 size  lastUpdated          downloadCount  
----------------------------------------------------------------  --------------------------------------------------  ------  -------------------  -------------  
christianlillelund/donald-trumps-rallies                          Donald Trump's Rallies                               720KB  2020-09-26 10:25:08            454  
heeraldedhia/groceries-dataset                                    Groceries dataset                                    257KB  2020-09-17 04:36:08           2536  
andrewmvd/trip-advisor-hotel-reviews                              Trip Advisor Hotel Reviews                             5MB  2020-09-30 08:31:20           1497  
balraj98/stanford-background-dataset                              Stanford Background Dataset                           17MB  2020-09-26 12:57:59            157  
nehaprabhavalkar/india

In [8]:
!kaggle datasets download -d grassknoted/asl-alphabet

100% 1.02G/1.03G [00:09<00:00, 103MB/s] 
100% 1.03G/1.03G [00:09<00:00, 118MB/s]


In [9]:
!unzip -q asl-alphabet.zip

In [27]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
from tensorflow.keras import layers
from google.colab import drive

In [11]:
!mkdir asl_alphabet_train_test/

In [12]:
!cp -r asl_alphabet_train/asl_alphabet_train/A/ asl_alphabet_train_test/
!cp -r asl_alphabet_train/asl_alphabet_train/B/ asl_alphabet_train_test/

In [15]:
data_dir="/content/asl_alphabet_train_test"
data_dir = pathlib.Path(data_dir)

In [18]:
batch_size = 32
img_height = 200
img_width = 200

In [19]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 6000 files belonging to 2 classes.
Using 4800 files for training.


In [20]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 6000 files belonging to 2 classes.
Using 1200 files for validation.


In [21]:
class_names = train_ds.class_names
print(class_names)

['A', 'B']


In [22]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [23]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [24]:
num_classes = 2

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [25]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [26]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)

Epoch 1/3
150/150 [==============================] - 36s 239ms/step - loss: 0.4834 - accuracy: 0.7356 - val_loss: 0.2845 - val_accuracy: 0.8883
Epoch 2/3
150/150 [==============================] - 37s 244ms/step - loss: 0.1065 - accuracy: 0.9604 - val_loss: 0.0720 - val_accuracy: 0.9825
Epoch 3/3
150/150 [==============================] - 35s 235ms/step - loss: 0.0400 - accuracy: 0.9867 - val_loss: 0.0332 - val_accuracy: 0.9883


In [42]:
#drive.flush_and_unmount()
drive.mount('/content/gdrive')
#drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [48]:
test_dir="/content/gdrive/My Drive/Capstone/Dataset/Jing"
test_dir = pathlib.Path(test_dir)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 200 files belonging to 2 classes.


In [49]:
test_loss, test_acc = model.evaluate(test_ds, verbose=1)

7/7 [==============================] - 0s 31ms/step - loss: 1.7008 - accuracy: 0.5750
